In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

train_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\train', transform=transform)
test_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\test', transform=transform)
val_data = datasets.ImageFolder('C:\\Users\\srisi\\OneDrive\\Desktop\\sidxt\\hub\\FractureCNN\\kaggle\\dataset\\val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)


In [3]:
conv1 = nn.Conv2d(1, 32, 3)
conv2 = nn.Conv2d(32, 64, 3)
conv3 = nn.Conv2d(64, 128, 3)
fc1 = nn.Linear(128 * 26 * 26, 256)
fc2 = nn.Linear(256, 128)
fc3 = nn.Linear(128, 1)
pool = nn.MaxPool2d(2)
dropout = nn.Dropout(0.3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv1.to(device)
conv2.to(device)
conv3.to(device)
fc1.to(device)
fc2.to(device)
fc3.to(device)
pool.to(device)
dropout.to(device)


def forward(x):
    x = pool(F.relu(conv1(x)))
    x = dropout(x)
    x = pool(F.relu(conv2(x)))
    x = dropout(x)
    x = pool(F.relu(conv3(x)))
    x = dropout(x)
    x = x.view(-1, 128 * 26 * 26)
    x = F.relu(fc1(x))
    x = dropout(x)
    x = F.relu(fc2(x))
    x = dropout(x)
    x = torch.sigmoid(fc3(x))
    return x
criterion = nn.BCELoss()
optimizer = optim.Adam(list(conv1.parameters()) + list(conv2.parameters()) + list(conv3.parameters()) + 
                       list(fc1.parameters()) + list(fc2.parameters()) + list(fc3.parameters()), lr=0.001)

In [4]:
for epoch in range(1):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = forward(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch {epoch+1}, loss: {running_loss / len(train_loader)}")


epoch 1, loss: 0.6532549185808315
epoch 2, loss: 0.5291392156320025
epoch 3, loss: 0.3672824773663121
epoch 4, loss: 0.24283229958179386
epoch 5, loss: 0.18922070696112492
epoch 6, loss: 0.13695241775825734
epoch 7, loss: 0.10097742663487042
epoch 8, loss: 0.07919186132765094
epoch 9, loss: 0.06058281565875479
epoch 10, loss: 0.07311160678527671
epoch 11, loss: 0.05053697496042551
epoch 12, loss: 0.04427949229358098
epoch 13, loss: 0.04512076855496352
epoch 14, loss: 0.03331059674467745
epoch 15, loss: 0.02690515008415709


In [5]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        #print("predicted - "+str(predicted))
        #print("label - "+str(labels))
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print(f"test accuracy: {100 * correct / total}%")

test accuracy: 96.99248120300751%


In [6]:
torch.save({
    'conv1_state_dict': conv1.state_dict(),
    'conv2_state_dict': conv2.state_dict(),
    'conv3_state_dict': conv3.state_dict(),
    'fc1_state_dict': fc1.state_dict(),
    'fc2_state_dict': fc2.state_dict(),
    'fc3_state_dict': fc3.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'main.pth')

In [7]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from sklearn.metrics import precision_recall_fscore_support
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

print(f"Test Accuracy: {100 * correct / total}%")
print(f"Precision: {precision*100}")
print(f"Recall: {recall*100}")
print(f"F1 Score: {f1*100}")

Test Accuracy: 96.99248120300751%
Precision: 100.0
Recall: 96.0
F1 Score: 97.95918367346938
